In [ ]:
#!pip install sshtunnel
#!pip install pymongo

In [18]:
from sshtunnel import SSHTunnelForwarder
import pymongo

tunnel = SSHTunnelForwarder( # define the tunnel parameters
    "51.124.209.38",
    remote_bind_address=('127.0.0.1', 27017),
    ssh_pkey = "C:/ssh/DBAP_Azure_OpenSSH",
    ssh_username = "azureuser",
)

tunnel.start() # start the tunnel to the VM OS
print ("Tunnel connected")

## start the connection to MongoDB
session = pymongo.MongoClient('127.0.0.1', tunnel.local_bind_port) # tunnel.local_bind_port is an assigned blind port on the local machine
print ("MongoDB connected")

Tunnel connected
MongoDB connected


In [19]:
import pymongo
import pandas as pd
import json
#open a database
db = session['CarbonEmissions_Database']
# list all collections within the database
collections = db.list_collection_names() 

In [20]:
collections


['Ozone_Collection',
 'Methane_Collection',
 'Carbonmonoxide_Collection',
 'Nitrogendioxide_Collection']

In [21]:
#types of emissions
Gas_Types = ["Ozone", "Methane", "Carbon_Monoxide", "Nitrogen_Dioxide"]
#dictionary for gases
Table_Dictionary = dict(zip(collections, Gas_Types))
#labels for data from API
Data_Labels = ["average", "count", "max", "min", "standard deviation", "State", "Gas"]

#for loop to go through gases
for table in Gas_Types: 
    #creates a datafram for each of the gases
    vars()[table] = pd.DataFrame() 
    
temp_df = pd.DataFrame()

#loop through all the collections
for collection in collections: 
    
    #sets iteration for collections
    Current_Collection = db[collection] 
    
    #finds all documents in collection
    cursor = Current_Collection.find() 
    # name table as label rather than code
    table = Table_Dictionary[collection] 
    
    #loop through all documents within the collection
    for doc in cursor: 
        json_df = pd.json_normalize(doc)
        temp_df["DateTime"] = json_df["time.interval_start"]
        temp_df["State"] = json_df["value.State"]
        temp_df["Average"] = json_df["value.average"]
        temp_df["Count"] = json_df["value.count"]
        temp_df["Max"] = json_df["value.max"]
        temp_df["Min"] = json_df["value.min"]
        temp_df["Std_Dev"] = json_df["value.standard deviation"]
        temp_df["DateTime"] = pd.to_datetime(temp_df["DateTime"])
        #concatenate temporary dataframes to final dataframes
        vars()[table] = pd.concat([vars()[table], temp_df], ignore_index = True, axis=0)
        
    #sets index to "DateTime"
    vars()[table].set_index('DateTime', inplace=True)
    vars()[table].sort_index(inplace=True)

In [22]:
Carbon_Monoxide.sort_index(inplace=True)
Carbon_Monoxide

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-01-01 06:00:00+00:00,Michigan,0.036024,1,0.036024,0.036024,None
2019-01-01 07:00:00+00:00,Michigan,0.038118,1,0.038118,0.038118,None
2019-01-01 07:00:00+00:00,Louisiana,0.012292,1,0.012292,0.012292,None
2019-01-01 07:00:00+00:00,Kentucky,0.023073,1,0.023073,0.023073,None
2019-01-01 17:00:00+00:00,Oregon,0.031744,1,0.031744,0.031744,None
...,...,...,...,...,...,...
2021-12-31 20:00:00+00:00,Utah,0.027735,1,0.027735,0.027735,None
2021-12-31 20:00:00+00:00,Alabama,0.029730,1,0.029730,0.029730,None
2021-12-31 20:00:00+00:00,Montana,0.027693,1,0.027693,0.027693,None


In [23]:
Nitrogen_Dioxide.sort_index(inplace=True)
Nitrogen_Dioxide

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-01-01 07:00:00+00:00,Louisiana,0.000010,1,0.000010,0.000010,None
2019-01-01 07:00:00+00:00,Michigan,0.000174,1,0.000174,0.000174,None
2019-01-01 08:00:00+00:00,Louisiana,0.000008,1,0.000008,0.000008,None
2019-01-01 08:00:00+00:00,Michigan,0.000208,1,0.000208,0.000208,None
2019-01-01 08:00:00+00:00,Kentucky,0.000011,1,0.000011,0.000011,None
...,...,...,...,...,...,...
2021-12-31 20:00:00+00:00,Iowa,0.000098,1,0.000098,0.000098,None
2021-12-31 20:00:00+00:00,New_Hampshire,0.000019,1,0.000019,0.000019,None
2021-12-31 20:00:00+00:00,Georgia,0.000009,1,0.000009,0.000009,None


In [24]:
Methane.sort_index(inplace=True)
Methane

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-01-01 07:00:00+00:00,Kentucky,1820.328247,1,1820.328247,1820.328247,None
2019-01-01 17:00:00+00:00,South_Dakota,1841.662598,1,1841.662598,1841.662598,None
2019-01-01 17:00:00+00:00,Connecticut,1836.352905,1,1836.352905,1836.352905,None
2019-01-01 19:00:00+00:00,Colorado,1794.046875,1,1794.046875,1794.046875,None
2019-01-01 19:00:00+00:00,New_Hampshire,1857.332520,1,1857.332520,1857.332520,None
...,...,...,...,...,...,...
2021-12-29 08:00:00+00:00,Kentucky,1877.454346,1,1877.454346,1877.454346,None
2021-12-30 22:00:00+00:00,California,1879.013184,1,1879.013184,1879.013184,None
2021-12-31 08:00:00+00:00,Kentucky,1863.877441,1,1863.877441,1863.877441,None


In [25]:
Ozone.sort_index(inplace=True)
Ozone

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-04-23 17:00:00+00:00,Ohio,0.144055,1,0.144055,0.144055,None
2019-04-23 17:00:00+00:00,South_Dakota,0.141773,1,0.141773,0.141773,None
2019-04-23 17:00:00+00:00,Delaware,0.148304,1,0.148304,0.148304,None
2019-04-23 17:00:00+00:00,Tennessee,0.141941,1,0.141941,0.141941,None
2019-04-23 17:00:00+00:00,Wisonsin,0.142006,1,0.142006,0.142006,None
...,...,...,...,...,...,...
2021-12-31 21:00:00+00:00,California,0.128483,1,0.128483,0.128483,None
2021-12-31 21:00:00+00:00,New_Jersey,0.136636,1,0.136636,0.136636,None
2021-12-31 21:00:00+00:00,Idaho,0.167380,1,0.167380,0.167380,None


In [26]:
# Test for duplicated column values and return those duplicates
# Ignores the index/datetime
Methane[Methane.duplicated(keep = False)].sort_values(by = "State")
# No real duplicates as same observations but different dates

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-04-28 18:00:00+00:00,Utah,1865.137939,1,1865.137939,1865.137939,None
2019-09-06 18:00:00+00:00,Utah,1865.137939,1,1865.137939,1865.137939,None


In [27]:
Carbon_Monoxide[Carbon_Monoxide.duplicated(keep = False)].sort_values(by = "State")

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-08-05 18:00:00+00:00,Minnesota,0.032450,1,0.032450,0.032450,None
2020-02-16 19:00:00+00:00,Minnesota,0.032450,1,0.032450,0.032450,None
2019-09-10 19:00:00+00:00,New_Hampshire,0.027668,1,0.027668,0.027668,None
2020-12-09 21:00:00+00:00,New_Hampshire,0.027668,1,0.027668,0.027668,None
2019-05-18 18:00:00+00:00,Tennessee,0.037292,1,0.037292,0.037292,None
2019-09-19 18:00:00+00:00,Tennessee,0.029253,1,0.029253,0.029253,None
2020-05-15 18:00:00+00:00,Tennessee,0.037292,1,0.037292,0.037292,None
2020-06-20 18:00:00+00:00,Tennessee,0.029253,1,0.029253,0.029253,None
2020-08-08 18:00:00+00:00,Utah,0.033722,1,0.033722,0.033722,None


In [28]:
Ozone[Ozone.duplicated(keep = False)].sort_values(by = "State")

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2019-11-29 07:00:00+00:00,Kentucky,0.145837,1,0.145837,0.145837,None
2020-02-16 07:00:00+00:00,Kentucky,0.145837,1,0.145837,0.145837,None
2019-09-18 20:00:00+00:00,Missouri,0.121547,1,0.121547,0.121547,None
2019-12-14 19:00:00+00:00,Missouri,0.121547,1,0.121547,0.121547,None
2020-01-15 17:00:00+00:00,New_York,0.144960,1,0.144960,0.144960,None
2020-09-20 20:00:00+00:00,New_York,0.144960,1,0.144960,0.144960,None
2019-09-27 20:00:00+00:00,North_Carolina,0.125004,1,0.125004,0.125004,None
2020-09-19 22:00:00+00:00,North_Carolina,0.125004,1,0.125004,0.125004,None
2020-05-26 18:00:00+00:00,Ohio,0.131596,1,0.131596,0.131596,None


In [29]:
Nitrogen_Dioxide[Nitrogen_Dioxide.duplicated(keep = False)].sort_values(by = "State")

,State,Average,Count,Max,Min,Std_Dev
DateTime,,,,,,
2020-08-02 20:00:00+00:00,Florida,0.000029,1,0.000029,0.000029,None
2021-05-25 21:00:00+00:00,Florida,0.000029,1,0.000029,0.000029,None
2021-02-18 00:00:00+00:00,Hawaii,0.000002,1,0.000002,0.000002,None
2021-07-22 01:00:00+00:00,Hawaii,0.000002,1,0.000002,0.000002,None


In [ ]:
# FALSE DUPLICATES RETURNED
#group by date and counts the number of observations
result = Methane.groupby(Methane.index.tolist(),as_index=False).size()
#returns all dates with 2 observations
print(result.loc[result["size"] > 1])

In [20]:
#Unnecessary to drop observations as there are no duplicates
for table in Gas_Types:
    vars()[table].index.duplicated()

In [30]:
for table in Gas_Types:
    print(vars()[table].tail())

                                State   Average  Count       Max       Min  \
DateTime                                                                     
2021-12-31 21:00:00+00:00  California  0.128483      1  0.128483  0.128483   
2021-12-31 21:00:00+00:00  New_Jersey  0.136636      1  0.136636  0.136636   
2021-12-31 21:00:00+00:00       Idaho  0.167380      1  0.167380  0.167380   
2021-12-31 21:00:00+00:00    Colorado  0.123041      1  0.123041  0.123041   
2021-12-31 21:00:00+00:00    Virginia  0.130226      1  0.130226  0.130226   

                          Std_Dev  
DateTime                           
2021-12-31 21:00:00+00:00    None  
2021-12-31 21:00:00+00:00    None  
2021-12-31 21:00:00+00:00    None  
2021-12-31 21:00:00+00:00    None  
2021-12-31 21:00:00+00:00    None  
                                State      Average  Count          Max  \
DateTime                                                                 
2021-12-29 08:00:00+00:00    Kentucky  1877.454346   

In [31]:
Carbon_Monoxide.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44359 entries, 2019-01-01 06:00:00+00:00 to 2021-12-31 20:00:00+00:00
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   State    44359 non-null  object 
 1   Average  44359 non-null  float64
 2   Count    44359 non-null  int64  
 3   Max      44359 non-null  float64
 4   Min      44359 non-null  float64
 5   Std_Dev  3 non-null      object 
dtypes: float64(3), int64(1), object(2)
memory usage: 2.4+ MB


In [32]:
import numpy as np
Carbon_Monoxide.describe()

,Average,Count,Max,Min
count,44359.000000,44359.000000,44359.000000,44359.000000
mean,0.031311,1.000068,0.031311,0.031311
std,0.006830,0.008224,0.006830,0.006830
min,0.011173,1.000000,0.011173,0.011173
25%,0.027608,1.000000,0.027608,0.027608
50%,0.031386,1.000000,0.031386,0.031386
75%,0.034867,1.000000,0.034867,0.034867
max,0.283208,2.000000,0.283208,0.283208


In [ ]:
%store Carbon_Monoxide
%store Nitrogen_Dioxide
%store Methane
%store Ozone
%store Data_Labels
%store Gas_Types

In [17]:
tunnel.close() # close the tunnel